In [42]:
import pandas as pd
from utils.load_data import LoadData 

### ground truth

In [43]:
# the ground truth values
load_data = LoadData()
data = load_data.data
data[data['year'] == 2024]

Data lost:  31


,year,month,day,week_day,consumo_max_diario,prod_e,prod_l,Consumo Vivix,MWh/dia,boosting,cor,espessura,extracao_forno,porcentagem_caco,_merge,crossed_threshold
365,2024,1,1,0,6.357671,0.0,0.0,146858.000,6.119117,1.8000,incolor,8.0,801.888266,12.969388,both,0
366,2024,1,2,1,6.353846,0.0,0.0,125399.000,5.224966,1.8000,incolor,6.0,760.639555,13.015363,both,0
367,2024,1,3,2,6.790784,0.0,0.0,142237.000,5.922399,1.8000,incolor,4.0,756.407065,12.955987,both,0
368,2024,1,4,3,7.210391,0.0,0.0,157797.000,6.449902,1.8000,incolor,3.0,751.027689,13.048787,both,0
369,2024,1,5,4,7.028001,1.0,0.0,156202.000,6.508437,1.8000,incolor,3.0,745.614081,13.009411,both,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2024,3,27,2,8.834712,0.0,0.0,194612.429,8.108851,3.0004,verde,8.0,807.694566,57.992219,both,1
452,2024,3,28,3,8.683421,0.0,0.0,194184.079,8.091003,3.0044,verde,8.0,827.881233,57.979331,both,1
453,2024,3,29,4,9.024435,0.0,1.0,203015.788,8.458991,3.0059,verde,8.0,825.910154,57.996623,both,1
454,2024,3,30,5,9.086995,0.0,1.0,207019.308,8.625804,3.0014,verde,8.0,825.581337,58.019722,both,1


In [44]:
demanda_2024 = data.loc[data['year'] == 2024][['year','month','day','crossed_threshold','consumo_max_diario',]]
demanda_2024['datetime'] = pd.to_datetime(demanda_2024[['year','month','day']])
demanda_2024 = demanda_2024.set_index('datetime')
demanda_2024.drop(columns=['year','month','day'], inplace=True)
demanda_2024.rename(columns={'crossed_threshold':'estouro de demanda', 'consumo_max_diario':'demanda (MWh)'}, inplace=True)
demanda_2024

,estouro de demanda,demanda (MWh)
datetime,,
2024-01-01,0,6.357671
2024-01-02,0,6.353846
2024-01-03,0,6.790784
2024-01-04,0,7.210391
2024-01-05,0,7.028001
...,...,...
2024-03-27,1,8.834712
2024-03-28,1,8.683421
2024-03-29,1,9.024435


# Xgboost

In [45]:
prediction_data_path = 'results/output_data/xgboost_2024-09-11_09-21-24.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data.index = pd.to_datetime(prediction_data['datetime'])
prediction_data.drop(columns=['datetime'], inplace=True)
prediction_data = prediction_data[['probabilidade_de_estouro', 'estouro_previsto']]
prediction_data

,probabilidade_de_estouro,estouro_previsto
datetime,,
2024-01-01,0.005157,0
2024-01-02,0.004496,0
2024-01-03,0.006817,0
2024-01-04,0.004897,0
2024-01-05,0.006186,0
...,...,...
2024-12-27,0.940286,1
2024-12-28,0.905265,1
2024-12-29,0.839121,1


In [49]:
results_df = pd.merge(prediction_data, demanda_2024, how='inner', left_index=True, right_index=True)
results_df

,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-01-01,0.005157,0,0,6.357671
2024-01-02,0.004496,0,0,6.353846
2024-01-03,0.006817,0,0,6.790784
2024-01-04,0.004897,0,0,7.210391
2024-01-05,0.006186,0,0,7.028001
...,...,...,...,...
2024-03-27,0.980687,1,1,8.834712
2024-03-28,0.977460,1,1,8.683421
2024-03-29,0.986550,1,1,9.024435


In [54]:
false_negative_error_df =results_df[(results_df['estouro_previsto'] == 0) & (results_df['estouro de demanda'] == 1)]
display(false_negative_error_df)
print(f'False negative error: {len(false_negative_error_df)} days of {len(results_df)} days')

,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,
2024-03-11,0.026346,0,1,8.802856
2024-03-13,0.205438,0,1,8.534767
2024-03-14,0.092298,0,1,8.879012
2024-03-15,0.092298,0,1,8.761183
2024-03-24,0.203780,0,1,8.745083
2024-03-25,0.007916,0,1,8.676560
2024-03-26,0.009029,0,1,9.144856


False negative error: 7 days of 91 days


In [55]:
false_positive_error_df =results_df[(results_df['estouro_previsto'] == 1) & (results_df['estouro de demanda'] == 0)]
display(false_positive_error_df)
print(f'False negative error: {len(false_positive_error_df)} days of {len(results_df)} days')

,probabilidade_de_estouro,estouro_previsto,estouro de demanda,demanda (MWh)
datetime,,,,


False negative error: 0 days of 91 days


### Xgboost without energy